# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [69]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
import re

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import pickle

In [13]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')

df = pd.read_sql_table("messages", engine)
X = df["message"].values
y = df.loc[:, "related":"direct_report"]

In [61]:
df.to_csv("disaster.csv")

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    translator = str.maketrans('', '', string.punctuation)
    doc = text.translate(translator)
    
    tokens = doc.split(" ")
    return tokens

In [63]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if bool(re.search(r"[^a-zA-Z0-9]", w)) != True]
    tokens = [WordNetLemmatizer().lemmatize(w, pos='v') for w in tokens if stopwords.words("english")]
    tokens = [PorterStemmer().stem(w) for w in tokens]
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [94]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True, tokenizer=tokenize, ngram_range=(1, 2), max_df=0.8, min_df=0.005)

In [95]:
matrix = tfidf_vectorizer.fit_transform(X)

In [96]:
matrix.shape

(26177, 886)

### For text scatter visualization 

In [97]:
tfidf_vectorizer.get_feature_names()

['1',
 '10',
 '100',
 '12',
 '15',
 '2',
 '20',
 '3',
 '30',
 '4',
 '4636',
 '5',
 '50',
 '8',
 'a',
 'a job',
 'a lot',
 'a tent',
 'abl',
 'abl to',
 'about',
 'about the',
 'access',
 'access to',
 'accord',
 'accord to',
 'aceh',
 'across',
 'across the',
 'activ',
 'add',
 'addit',
 'address',
 'affect',
 'affect by',
 'afghanistan',
 'afp',
 'africa',
 'after',
 'after the',
 'aftershock',
 'again',
 'against',
 'agenc',
 'agricultur',
 'aid',
 'air',
 'all',
 'all the',
 'along',
 'alreadi',
 'also',
 'american',
 'among',
 'an',
 'and',
 'and a',
 'and be',
 'and food',
 'and i',
 'and in',
 'and it',
 'and other',
 'and the',
 'and to',
 'and water',
 'and we',
 'ani',
 'anoth',
 'answer',
 'anyth',
 'april',
 'area',
 'area of',
 'arm',
 'armi',
 'around',
 'arriv',
 'as',
 'as a',
 'as the',
 'as well',
 'ask',
 'ask for',
 'assess',
 'assist',
 'at',
 'at least',
 'at the',
 'attack',
 'author',
 'avail',
 'away',
 'babi',
 'back',
 'bank',
 'base',
 'basic',
 'be',
 'be a'

In [77]:
from sklearn.decomposition import PCA

In [100]:
pca = PCA(n_components=2)
transformed_centroid = pca.fit_transform(matrix.toarray())

In [103]:
text_cluster = dict(zip(X, transformed_centroid))

In [104]:
with open("text_cluster.pkl", "wb") as outfile:
    pickle.dump(text_cluster, outfile)

### Building a classifier

In [8]:
multi_obj = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)

In [33]:
multi_obj.fit(matrix, y.values)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1)

In [54]:
for m, pred in zip(y.columns, multi_obj.predict(tfidf_vectorizer.transform(["We are dire need of medical help! First aid"])).squeeze()):
    print(m, pred)

related 1
request 0
offer 0
aid_related 1
medical_help 0
medical_products 0
search_and_rescue 0
security 0
military 0
child_alone 0
water 0
food 0
shelter 0
clothing 0
money 0
missing_people 0
refugees 0
death 0
other_aid 0
infrastructure_related 0
transport 0
buildings 0
electricity 0
tools 0
hospitals 0
shops 0
aid_centers 0
other_infrastructure 0
weather_related 0
floods 0
storm 0
fire 0
earthquake 0
cold 0
other_weather 0
direct_report 0


In [5]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(lowercase=True, tokenizer=tokenize, stop_words=stopwords.words("english"))),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
])

In [10]:
pipeline.fit(X.values, y.values)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

In [11]:
pipeline.score(X.values, y.values)

0.81384421438667531

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

In [14]:
pipeline.score(X_test, y_test)

0.23815889992360581

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
with open("model-adaboost.pkl", "rb") as infile:
    model = pickle.load(infile)

/home/niraj/.virtualenvs/text/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  """Set the parameters of this estimator.
/home/niraj/.virtualenvs/text/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  """Set the parameters of this estimator.
/home/niraj/.virtualenvs/text/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  """Set the parameters of this estimator.
/home/niraj/.virtualenvs/text/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning

In [20]:
y_pred = model.predict(X_test)
y_pred

array([[1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [21]:
# for pred, true in zip(y_pred, y_test):
#     classification_report(pred[0], true[0])

for i in range(36):
    pred = [x[i] for x in y_pred]
    true = [x[i] for x in y_test.values]
    print("\n\nClass: {}".format(y.columns[i]))
    print(classification_report(pred, true))



Class: related
              precision    recall  f1-score   support

           0       0.25      0.58      0.35       533
           1       0.94      0.80      0.86      4679
           2       0.50      0.83      0.62        24

   micro avg       0.77      0.77      0.77      5236
   macro avg       0.56      0.74      0.61      5236
weighted avg       0.87      0.77      0.81      5236



Class: request
              precision    recall  f1-score   support

           0       0.97      0.92      0.95      4616
           1       0.58      0.80      0.67       620

   micro avg       0.91      0.91      0.91      5236
   macro avg       0.78      0.86      0.81      5236
weighted avg       0.93      0.91      0.91      5236



Class: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5227
           1       0.17      0.44      0.25         9

   micro avg       1.00      1.00      1.00      5236
   macro avg       0.59  

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5211
           1       0.33      0.76      0.46        25

   micro avg       0.99      0.99      0.99      5236
   macro avg       0.67      0.88      0.73      5236
weighted avg       1.00      0.99      0.99      5236



Class: earthquake
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4799
           1       0.79      0.87      0.83       437

   micro avg       0.97      0.97      0.97      5236
   macro avg       0.89      0.92      0.91      5236
weighted avg       0.97      0.97      0.97      5236



Class: cold
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5179
           1       0.42      0.79      0.55        57

   micro avg       0.99      0.99      0.99      5236
   macro avg       0.71      0.89      0.77      5236
weighted avg       0.99      0.99      0

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    'vect__ngram_range': ((1,1), (1,2)),
    'vect__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
for i in range(36):
    pred = [x[i] for x in y_pred]
    true = [x[i] for x in y_test]
    print("\n\nClass: {}".format(y.columns[i]))
    print(classification_report(pred, true))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open("baseline.pkl", "wb") as outfile:
    pickle.dump(cv, outfile)

## 10. Statistics to visualize

In [26]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [27]:
df_categories = df.drop(["id", "message", "original", "genre"], axis=1)

In [38]:
df_categories.related.value_counts()

6115

In [46]:
help_counts = [df[column].value_counts()[1] if 1 in df[column].unique() else 0 for column in df_categories.columns]

In [59]:
help_dict = sorted(dict(zip(df_categories.columns, help_counts)).items(), key=lambda x: x[1], reverse=True)
help_dict

[('related', 19874),
 ('aid_related', 10840),
 ('weather_related', 7286),
 ('direct_report', 5063),
 ('request', 4464),
 ('other_aid', 3441),
 ('food', 2917),
 ('earthquake', 2452),
 ('storm', 2440),
 ('shelter', 2308),
 ('floods', 2149),
 ('medical_help', 2080),
 ('infrastructure_related', 1705),
 ('water', 1669),
 ('other_weather', 1376),
 ('buildings', 1331),
 ('medical_products', 1310),
 ('transport', 1199),
 ('death', 1192),
 ('other_infrastructure', 1151),
 ('refugees', 874),
 ('military', 859),
 ('search_and_rescue', 724),
 ('money', 603),
 ('electricity', 532),
 ('cold', 528),
 ('security', 471),
 ('clothing', 404),
 ('aid_centers', 309),
 ('missing_people', 298),
 ('hospitals', 283),
 ('fire', 282),
 ('tools', 159),
 ('shops', 120),
 ('offer', 117),
 ('child_alone', 0)]

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.